## 그랩컷 (GrabCut)

그래프 컷(graph cut) 기반 **영역 분할** 알고리즘

영상의 픽셀을 그래프의 정점(node)으로 간주하고, 픽셀들을 두 개의 그룹(객체/배경)으로 나누는 최적의 컷을 찾는 방식 (Max Flow Minimum Cut)

사용자가 지정한 전경/배경 정보를 활용하여 영상 분할

**그래프 컷**

그래프가 존재할 때 그래프의 정점들을 두 개의 서브 집합으로 나누는 segmenting 작업

![image](https://user-images.githubusercontent.com/44194558/143730767-fa4c0cab-be35-486b-a543-4cbbfbede763.png)

**그랩컷**

그래프 컷 기반 영역 분할 알고리즘

![image](https://user-images.githubusercontent.com/44194558/143730791-0520cb08-0065-4f92-914a-202c153c7f57.png)


<br/>

![image](https://user-images.githubusercontent.com/44194558/143730802-5cdd1d55-2a1e-4144-871f-de387bfa6c21.png)


**그랩컷 함수**

cv2.grabCut(img, mask, rect, bgdModel, fgdModel, iterCount, mode=None) -> mask, bgdModel, fgdModel

`img` : 입력 영상

`mask` : 입출력 마스크 (GC_BGD(0), GC_FGD(1), GC_PR_BGD(2), GC_PR_FGD(3))
  - 0~3의 값 만을 갖는 행렬
  - BGD/FGD : 배경/전경, PR : 아마도~일 것이다

`rect` : ROI 영역

`bgdModel` : 임시 배경 모델 행렬. 같은 영상 처리 시에는 변경 금지

`fgdModel` : 임시 전경 모델 행렬. 같은 영상 처리 시에는 변경 금지

  - grabCut 함수는 segmenting 한 결과를 가지고 또 다시 segmenting하여 최적의 결과를 출력
  - bgdModel, fgdModel을 지정하고 이를 이용해 recursive하게 segmenting 구현

`iterCount` : 결과 생성을 위한 Segmenting 반복 횟수

`mode` : cv2.GC_로 시작하는 모드 상수. cv2.GC_INIT_WITH_RECT 또는 cv2.GC_INIT_WITH_MASK

In [9]:
import sys
import numpy as np
import cv2

src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch08/nemo.jpg')

if src is None:
    print('Image load failed!')
    sys.exit()

# 사각형 지정을 통해 객체를 지정
rc = cv2.selectROI(src)  # 영상에서 객체에 해당하는 부분을 사용자로부터 사각형 모양으로 지정 받아, 해당 사각형에 대한 정보를 반환
print('rc', rc)
mask = np.zeros(src.shape[:2], np.uint8)  # 입력 영상과 동일한 크기로 마스크 영상 초기화

cv2.grabCut(src, mask, rc, None, None, 5, cv2.GC_INIT_WITH_RECT)  # mask에 배경 값, 배경일 거 같은 값, 객체 값 등등 자동으로 지정

# 0: cv2.GC_BGD, 2: cv2.GC_PR_BGD
mask2 = np.where((mask == 0) | (mask == 2), 0, 1).astype('uint8')   # mask에 배경인 부분(0), 배경일 것 같은 부분(2)을 검정색(0)으로 지정, 나머지는 하얀색(1)으로 지정
print('mask2 shape', mask2.shape)
print('src shape', src.shape)

# mask 연산
dst = src * mask2[:, :, np.newaxis]  # newaxis를 적용하여 동일한 차원으로 만든 후 곱셈 (객체를 제외한 부분은 모두 0, 검정색)

# 초기 분할 결과 출력
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

rc (145, 155, 360, 194)
mask2 shape (426, 640)
src shape (426, 640, 3)


**mask 영상**

![image](https://user-images.githubusercontent.com/44194558/143731147-8db7af8b-c571-496d-bd22-a30dfc3408d6.png)


- 배경 부분은 검정색, 배경일 것 같은 부분은 회색, 객체 부분은 흰색에 가까운 회색

In [18]:
import sys
import numpy as np
import cv2
from skimage import io

src = io.imread('messi5.jpg')

if src is None:
    print('Image load failed!')
    sys.exit()
    
# 사각형 지정을 통한 초기 분할
mask = np.zeros(src.shape[:2], np.uint8)  # 마스크
bgdModel = np.zeros((1, 65), np.float64)  # 배경 모델 ((1, 65) 형태로 미리 생성)
fgdModel = np.zeros((1, 65), np.float64)  # 전경 모델

rc = cv2.selectROI(src)

cv2.grabCut(src, mask, rc, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)

# 0: cv2.GC_BGD, 2: cv2.GC_PR_BGD
mask2 = np.where((mask == 0) | (mask == 2), 0, 1).astype('uint8')
dst = src * mask2[:, :, np.newaxis]

# 초기 분할 결과 출력
cv2.imshow('dst', dst)

# 마우스 이벤트 처리 함수 등록
def on_mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # 왼쪽 버튼은 전경
        cv2.circle(dst, (x, y), 3, (255, 0, 0), -1)  # 파랑색 색칠
        cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)  # 마스크에 전경 강제 
        cv2.imshow('dst', dst)
    elif event == cv2.EVENT_RBUTTONDOWN:  # 오른쪽 버튼은 배경
        cv2.circle(dst, (x, y), 3, (0, 0, 255), -1)  # 빨강색 원
        cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)  # 마스크에 배경 강제 
        cv2.imshow('dst', dst)
    elif event == cv2.EVENT_MOUSEMOVE:  # 마우스 움직임
        if flags & cv2.EVENT_FLAG_LBUTTON:  # 왼쪽 누르고 움직이면 전경
            cv2.circle(dst, (x, y), 3, (255, 0, 0), -1)
            cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)
            cv2.imshow('dst', dst)
        elif flags & cv2.EVENT_FLAG_RBUTTON:  # 오른쪽 누르고 움직이면 배경
            cv2.circle(dst, (x, y), 3, (0, 0, 255), -1)
            cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)
            cv2.imshow('dst', dst)
            
cv2.setMouseCallback('dst', on_mouse)

while True:
    key = cv2.waitKey()
    if key == 13:  # ENTER
        # 사용자가 지정한 전경/배경 정보를 활용하여 영상 분할
        cv2.grabCut(src, mask, rc, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_MASK)
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
        dst = src * mask2[:, :, np.newaxis]
        cv2.imshow('dst', dst)
    elif key == 27:
        break

cv2.destroyAllWindows()

## 모멘트 기반 객체 검출

**모멘트**

**영상의 모양, 형태 정보를 표현**하는 일련의 실수값들의 집합으로 해당 영상의 특징 벡터를 추출하는 방법

특정 함수 집합과의 상관 관계 형태로 계산

<br/>

**Hu의 7개 불변 모멘트**

정규화된 중심 모멘트 값을 조합하여 만든 7개의 모멘트 값

**영상의 크기, 회전, 이동, 대칭 변환에 불변**하기 때문에 두 영상에서 원하는 객체를 비교, 인식 할 때 사용

**모양 비교 함수**

cv2.matchShapes(contour1, contour2, method, parameter) -> retval

`contour1` : 첫 번째 외곽선 또는 그레이스케일 영상

`contour2` : 두 번째 외곽선 또는 그레이스케일 영상

`method` : 비교 방법 지정 (CONTOURS_MATCH_I1 / MATCH_I2 / MATCH_I3)

![image](https://user-images.githubusercontent.com/44194558/143732551-58b24fd5-fc96-4f6b-80a2-bfac44259b14.png)

`parameter` : 0

`retval` : 두 외곽선 또는 그레이 스케일 영상 간의 벡터 거리

    - 두 객체가 비슷하면 특징을 나타내는 모멘트 벡터의 값이 유사하여 값의 차이가 작음, 다를 수록 벡터 값이 다르기 때문에 차이가 크게 나타남

HU의 불변 모멘트를 사용하여 두 외곽선 또는 영상의 모양을 비교


In [25]:
import sys
import numpy as np
import cv2

# symbols에서 spade 찾기
obj = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch08/spades.png', cv2.IMREAD_GRAYSCALE)  # 스페이드
src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch08/symbols.png', cv2.IMREAD_GRAYSCALE)  # 스페이드를 비롯한 다양한 도형들

if src is None or obj is None:
    print('Image load failed!')
    sys.exit()

# 객체 영상 외곽선 검출
_, obj_bin = cv2.threshold(obj, 128, 255, cv2.THRESH_BINARY_INV)  # 영상의 색을 반전 (검은색 도형들), 이진화
obj_contours, _ = cv2.findContours(obj_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # 외곽선 검출 (바깥 윤곽선, 모든 외곽선 좌표 구하기)
obj_pts = obj_contours[0]

# 입력 영상 분석
_, src_bin = cv2.threshold(src, 128, 255, cv2.THRESH_BINARY_INV)
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # 모든 객체의 외곽선 정보를 저장

# 결과 영상
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

# 입력 영상의 모든 객체의 외곽선 정보에 대해서
# 입력 영상의 모든 객체 영역에 대해 contours에 있는 각각의 점들의 집합을 검출
for pts in contours:
    if cv2.contourArea(pts) < 1000:  # 너무 작은 노이즈 객체는 배제
        continue

    rc = cv2.boundingRect(pts)  # 외곽선의 좌표를 입력하면 사각형 좌표 반환
    cv2.rectangle(dst, rc, (255, 0, 0), 1)

    # 모양 비교
    dist = cv2.matchShapes(obj_pts, pts, cv2.CONTOURS_MATCH_I3, 0)

    cv2.putText(dst, str(round(dist, 4)), (rc[0], rc[1] - 3),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)

    if dist < 0.1:
        cv2.rectangle(dst, rc, (0, 0, 255), 2)

cv2.imshow('obj', obj)
cv2.imshow('dst', dst)
cv2.waitKey(0)

27

## 템플릿 매칭

**템플릿** : 찾을 대상이 되는 작은 영상. patch

입력 영상에서 작은 크기의 부분(템플릿) 영상과 일치하는 부분을 찾는 기법

작은 크기의 템플릿 영상을 입력 영상 전체 영역에 대해 이동하면서 가장 비슷한 위치를 수치적으로 찾아내는 방식

<br/>


![image](https://user-images.githubusercontent.com/44194558/143732907-2eaa9f4e-e205-4a40-8442-c118cc71bcb5.png)

- 템플릿 영상을 입력 영상의 전체 영역에 대해 이동하면서 템플릿 영상과 입력 영상 간의 유사도/비유사도를 계산

- 유사도는 템플릿 영상과 비슷한 부분 영상 위치에서 값이 크게 나타남


cv2.matchTemplate(image, temp1, method, result=None, mask=None) -> result

`image` : 입력 영상 (W x H)

`templ` : 템플릿 영상. image보다 같거나 작은 크기, 같은 타입 (w x h)

`method` : 비교 방법. cv2.TM_으로 시작하는 플래그 지정


![image](https://user-images.githubusercontent.com/44194558/143733067-4a31caa9-021a-42ae-9790-c9f93a15c480.png)

<br/>

![image](https://user-images.githubusercontent.com/44194558/143733218-058670a2-21e9-476c-ac7e-f9c5514ce322.png)


`result` : 비교 결과 행렬. numpy.ndarray. dtype=numpy.float32 (W - w + 1 x H - h + 1)
    

In [27]:
import sys
import numpy as np
import cv2

# 입력 영상 & 템플릿 영상
src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch08/circuit.bmp', cv2.IMREAD_GRAYSCALE)
templ = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch08/crystal.bmp', cv2.IMREAD_GRAYSCALE)

if src is None or templ is None:
    print('Image load failed!')
    sys.exit()

# 입력 영상 밝기 50증가, 가우시안 잡음(sigma=10) 추가
noise = np.zeros(src.shape, np.int32)
cv2.randn(noise, 50, 10)
src = cv2.add(src, noise, dtype=cv2.CV_8UC3)

# 템플릿 매칭 & 결과 분석
res = cv2.matchTemplate(src, templ, cv2.TM_CCOEFF_NORMED)
res_norm = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

# 최소/최대 위치 함수 (최소 포인터, 최대 포인터, 최소 지점, 최대 지점)
_, maxv, _, maxloc = cv2.minMaxLoc(res)
print('maxv:', maxv)
print('maxloc:', maxloc)  # 좌측 상단 모서리 좌표

# 매칭 결과를 빨간색 사각형으로 표시
th, tw = templ.shape[:2]
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
cv2.rectangle(dst, maxloc, (maxloc[0] + tw, maxloc[1] + th), (0, 0, 255), 2)

# 결과 영상 화면 출력
cv2.imshow('res_norm', res_norm)  # 유사도의 그레이 스케일 영상
cv2.imshow('dst', dst)  # 매칭 결과 영상
cv2.waitKey()
cv2.destroyAllWindows()

maxv: 0.9796523451805115
maxloc: (568, 320)


cv2.minMaxLoc(res)


- cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED는 최소 지점(minLoc)이 검출된 위치


- cv2.TM_CCORR, cv2.TM_CCORR_NORMED, cv2.TM_CCOEFF, cv2.TM_CCOEFF_NORMED는 최대 지점(maxLoc)이 검출된 위치

### 템플릿 매칭 - 인쇄체 숫자 인식


템플릿 영상 : Consolas 폰트로 쓰여진 0~9 숫자 영상 (100 x 150)

![image](https://user-images.githubusercontent.com/44194558/143733415-30e0b220-f3a2-42d1-a36b-993e960ccf36.png)

In [ ]:
import sys
import numpy as np
import cv2


# 숫자 템플릿 영상 digit0.bmp ~ digit9.bmp 불러오기
def load_digits():
    img_digits = []

    for i in range(10):
        filename = 'C:/Users/ky_moon/Desktop/vision/ch08/digits/digit{}.bmp'.format(i)
        img_digits.append(cv2.imread(filename, cv2.IMREAD_GRAYSCALE))

        if img_digits[i] is None:
            return None

    return img_digits


def find_digit(img, img_digits):
    max_idx = -1
    max_ccoeff = -1

    # 최대 NCC 찾기
    for i in range(10):
        img = cv2.resize(img, (100, 150))
        res = cv2.matchTemplate(img, img_digits[i], cv2.TM_CCOEFF_NORMED)

        if res[0, 0] > max_ccoeff:
            max_idx = i
            max_ccoeff = res[0, 0]

    return max_idx


def main():
    # 입력 영상 불러오기
    src = cv2.imread('digits_print.bmp')

    if src is None:
        print('Image load failed!')
        return

    # 100x150 숫자 템플릿 영상 불러오기
    img_digits = load_digits()  # list of ndarray

    if img_digits is None:
        print('Digit image load failed!')
        return

    # 입력 영상 이진화 & 레이블링
    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, src_bin = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(src_bin)  # 객체 개수, 바운딩 박스 정보

    # 숫자 인식 결과 영상 생성
    dst = src.copy()
    for i in range(1, cnt):
        (x, y, w, h, s) = stats[i]

        if s < 1000:
            continue

        # 가장 유사한 숫자 이미지를 선택
        digit = find_digit(src_gray[y:y+h, x:x+w], img_digits)
        cv2.rectangle(dst, (x, y, w, h), (0, 255, 255))
        cv2.putText(dst, str(digit), (x, y - 4), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()

## 캐스케이드 분류기 : 얼굴 검출

영상을 24 x 24 크기로 정규화한 후, 유사-하르 필터 (Haar-like) 집합으로부터 특징 정보를 추출하여 얼굴 여부를 판별

AdaBoost에 기반한 강한 분류 성능 & 캐스케이드 방식을 통한 빠른 동작

<br/>



**유사 하르 필터**

흑백 사각형이 서로 붙어 있는 형태로 구성된 필터

흰색 영역 픽셀 값은 모두 더하고, 검은색 영역 픽셀 값은 모두 빼서 하나의 특징 값 계산

유사 하르 필터로 구한 특징 값을 통해 얼굴을 판별

  - 얼굴 형태는 전형적으로 밝은 영역(이마, 미간, 볼 등), 어두운 영역(눈썹, 입술 등)으로 정해져 있음

![image](https://user-images.githubusercontent.com/44194558/143734100-730dc9b1-e4a0-487d-bace-910c2d6f0dab.png)

<br/>

![image](https://user-images.githubusercontent.com/44194558/143734183-894340db-9471-40ea-a36d-953c9f869d40.png)

**캐스케이드 분류기**


일반적인 영상에서 대부분의 영역이 non-face 영역

non face영역을 빠르게 skip하고 넘어갈 수 있도록 다단계 검사 수행


![image](https://user-images.githubusercontent.com/44194558/143734491-8d4754b7-d6a0-4f67-bc95-1875b0e69e16.png)


 - 특정 단계에서 얼굴이 아니라고 판단되면 더 이상 검사를 수행하지 않음, 통과하면 다음 단계로 pass 
 
 
참고 : https://www.youtube.com/watch?v=hPCTwxF0qf4

**객체 생성 및 학습 데이터 불러오기 함수 - cv2.CascadeClassifier()**

미리 학습된 정보를 불러와서 내가 찾고자 하는 객체를 검출

filename으로 저장된, 객체의 특징을 담고 있는 파일을 불러올 수 있음

cv2.CascadeClassifier( ) -> CascadeClassifier object    
cv2.CascadeClassifier(filename) -> CascadeClassifier object 

cv2.CascadeClassifier.load(filename) -> retval (filename을 지정했으면 load 할 필요가 x)

`filename` : XML 파일 이름

`retval` : 성공하면 True, 실패하면 False


미리 학습된 XML 파일 다운로드 : https://github.com/opencv/opencv/tree/master/data/haarcascades


![image](https://user-images.githubusercontent.com/44194558/143734651-59209441-f874-4906-9c4b-1fe0678018c1.png)

**CascadeClassifier 멀티 스케일 객체 검출 함수**

cv2.CascadeClassifier.detectMultiScale(image, scaleFactor=None, minNeighbors=None, flags=None, minSize=None, maxSize=None) -> result

`image` : 입력 영상 (cv2.CV_8U)

`scaleFactor` : 영상 축소 비율. 기본값은 1.1.

`minNeighbors` : 얼마나 많은 이웃 사각형이 검출되어야 최종 검출 영역으로 설정할지를 지정. 기본값은 3.

`flags` : (현재) 사용되지 않음

`minSize` : 최소 객체 크기. (w, h) 튜플.

`maxSize` : 최대 객체 크기. (w, h) 튜플.

`result` : 검출된 객체의 사각형 정보(x, y, w, h)를 담은 numpy.ndarray. shape=(N, 4). dtype=numpy.int32.

In [38]:
import sys
import numpy as np
import cv2

src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch08/lenna.bmp')

if src is None:
    print('Image load failed!')
    sys.exit()

classifier = cv2.CascadeClassifier('C:/Users/ky_moon/Desktop/vision/ch08/haarcascade_frontalface_alt2.xml')

if classifier.empty():
    print('XML load failed!')
    sys.exit()

faces = classifier.detectMultiScale(src)

for (x, y, w, h) in faces:
    cv2.rectangle(src, (x, y, w, h), (255, 0, 255), 2)

cv2.imshow('src', src)
cv2.waitKey()
cv2.destroyAllWindows()

In [39]:
faces

array([[217, 200, 179, 179]], dtype=int32)

## HOG 알고리즘 & 보행자 검출 

**Histogram of Oriented Gradients**

영상의 지역적 그래디언트 방향 정보를 특징 벡터로 사용

사람이 서 있는 영상에서 그래디언트를 구하고, 그래디언트의 크기와 방향 성분을 이용하여 사람이 서 있는 형태에 대한 특징 벡터를 정의

  - SVM을 이용하여 입력 영상에서 보행자 위치를 검출



![image](https://user-images.githubusercontent.com/44194558/143734934-d11653a2-ad27-463f-bc6b-81f9205c71c2.png)



1. 64 x 128 영상에서 계산


2. 입력 영상으로부터 그래디언트 계산 (크기, 방향 성분 0~180도)



3. 입력 영상을 8x8 크기 단위로 분할 (가로 8, 세로 16개)


4. 각각의 셀로부터 그래디언트 방향 성분에 대한 히스토그램 계산

   - 방향 성분을 20도 단위로 구분하여 총 9개의 bin으로 구성된 방향 히스토그램 구성
   
   
   
5. 인접한 4개의 셀을 합쳐서 블록이라고 정의

   - 하나의 블록에는 4개의 셀, 각 셀에는 9개의 bin으로 구성된 방향 히스토그램 정보
   
   - 블록 하나에는 총 36개의 실수 값으로 구성된 방향 히스토그램 정보
   
   
   
6. 블록은 가로, 세로 방향으로 각각 한 개의 셀 만큼 이동하면서 정의

   - 가로 방향으로 7개, 세로 방향으로 15개 정의 가능
   
   - 64 x 128 입력 영상에서 총 105개의 블록 추출, 전체 블록에서 추출되는 방향 히스토그램 실수 값은 105 x 36 = 3780개
   
   
7. 3780개의 실수 값이 64 x 128 영상을 표현하는 HOG 특징 벡터

**HOG 기술자 객체 생성 및 학습된 분류기 계수 불러오기 - cv2.HOGDescriptor**


cv2.HOGDescriptor()  ->  HOGDescriptor object

cv2.HOGDescriptor_getDefaultPeopleDetector() -> retval (미리 훈련된 특징 벡터 가져오기)

`retval` : 미리 훈련된 특징 벡터. numpy.ndarray. shape=(3781, 1). dtype=numpy.float32.

**HOG 멀티스케일 객체 검출 함수**

cv2.HOGDescriptor.detectMultiScale(img, hitThreshold=None, winStride=None, padding=None, scale=None, finalThreshold=None, useMeanshiftGrouping=None) -> foundLocations, foundWeights

`img` : 입력 영상. cv2.CV_8UC1 또는 cv2.CV_8UC3.

`hitThreshold` : 특징 벡터와 SVM 분류 평면까지의 거리에 대한 임계값

`winStride` : 셀 윈도우 이동 크기. (0, 0) 지정 시 셀 크기와 같게 설정.

`padding` : 패딩 크기

`scale` : 검색 윈도우 크기 확대 비율. 기본값은 1.05.

`finalThreshold` : 검출 결정을 위한 임계값

`useMeanshiftGrouping` : 겹쳐진 검색 윈도우를 합치는 방법 지정 플래그

`foundLocations` : (출력) 검출된 사각형 영역 정보

`foundWeights` : (출력) 검출된 사각형 영역에 대한 신뢰도

In [41]:
import sys
import random
import numpy as np
import cv2

cap = cv2.VideoCapture('C:/Users/ky_moon/Desktop/vision/ch08/vtest.avi')

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

# 보행자 검출을 위한 HOG 기술자 설정
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # 매 프레임마다 보행자 검출
    detected, _ = hog.detectMultiScale(frame)

    # 검출 결과 화면 표시
    for (x, y, w, h) in detected:
        c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        cv2.rectangle(frame, (x, y, w, h), c, 3)

    cv2.imshow('frame', frame)
    if cv2.waitKey(10) == 27:
        break

cv2.destroyAllWindows()

## SnowAPP

캐스케이드 분류기 사용

  - XML : https://github.com/opencv/opencv/tree/master/data/haarcascades
  
  - 얼굴 검출 XML : haarcascade_frontalface_alt2.xml
  
  - 눈 검출 XML : haarcascade_eye.xml
  
  
눈 검출

  - 얼굴 검출 영역 내에서만 눈 검출
  
  - 2개 검출되었을 때만 그래픽 합성
  
  - 양안의 눈 좌표를 계산

In [42]:
import sys
import numpy as np
import cv2


# 3채널 img 영상에 4채널 item 영상을 pos 위치에 합성
def overlay(img, glasses, pos):
    # 실제 합성을 수행할 부분 영상 좌표 계산
    sx = pos[0]  # start x
    ex = pos[0] + glasses.shape[1]  # start x + glasses.width
    sy = pos[1]  # start y
    ey = pos[1] + glasses.shape[0]  # start y + glasses.height

    # 합성할 영역이 입력 영상 크기를 벗어나면 무시
    if sx < 0 or sy < 0 or ex > img.shape[1] or ey > img.shape[0]:
        return

    # 부분 영상 참조. img1: 입력 영상의 부분 영상, img2: 안경 영상의 부분 영상
    img1 = img[sy:ey, sx:ex]   # shape=(h, w, 3)
    img2 = glasses[:, :, 0:3]  # shape=(h, w, 3)
    alpha = 1. - (glasses[:, :, 3] / 255.)  # shape=(h, w)

    # BGR 채널별로 두 부분 영상의 가중합
    img1[..., 0] = (img1[..., 0] * alpha + img2[..., 0] * (1. - alpha)).astype(np.uint8)
    img1[..., 1] = (img1[..., 1] * alpha + img2[..., 1] * (1. - alpha)).astype(np.uint8)
    img1[..., 2] = (img1[..., 2] * alpha + img2[..., 2] * (1. - alpha)).astype(np.uint8)


# 카메라 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Camera open failed!')
    sys.exit()

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('C:/Users/ky_moon/Desktop/vision/ch08/output.avi', fourcc, 30, (w, h))

# Haar-like XML 파일 열기
face_classifier = cv2.CascadeClassifier('C:/Users/ky_moon/Desktop/vision/ch08/haarcascade_frontalface_alt2.xml')
eye_classifier = cv2.CascadeClassifier('C:/Users/ky_moon/Desktop/vision/ch08/haarcascade_eye.xml')

if face_classifier.empty() or eye_classifier.empty():
    print('XML load failed!')
    sys.exit()

# 안경 PNG 파일 열기 (Image from http://www.pngall.com/)
glasses = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch08/glasses.png', cv2.IMREAD_UNCHANGED)

if glasses is None:
    print('PNG image open failed!')
    sys.exit()

ew, eh = glasses.shape[:2]  # 가로, 세로 크기
ex1, ey1 = 240, 300  # 왼쪽 눈 좌표
ex2, ey2 = 660, 300  # 오른쪽 눈 좌표

# 매 프레임에 대해 얼굴 검출 및 안경 합성
while True:
    ret, frame = cap.read()

    if not ret:
        break

    # 얼굴 검출
    faces = face_classifier.detectMultiScale(frame, scaleFactor=1.2,
                                             minSize=(100, 100), maxSize=(400, 400))

    for (x, y, w, h) in faces:
        #cv2.rectangle(frame, (x, y, w, h), (255, 0, 255), 2)

        # 얼굴 영역에서 눈 검출
        faceROI = frame[y:y + h // 2, x:x + w]
        eyes = eye_classifier.detectMultiScale(faceROI)

        # 눈을 2개 검출한 것이 아니라면 무시
        if len(eyes) != 2:
            continue

        # 두 개의 눈 중앙 위치를 (x1, y1), (x2, y2) 좌표로 저장
        # 눈의 좌표는 얼굴 영상에서 계산된 것이기 때문에 전체 영상의 눈 좌표를 계산 (가로, 세로 크기의 반을 추가적으로 더해줌)
        x1 = x + eyes[0][0] + (eyes[0][2] // 2)
        y1 = y + eyes[0][1] + (eyes[0][3] // 2)
        x2 = x + eyes[1][0] + (eyes[1][2] // 2)
        y2 = y + eyes[1][1] + (eyes[1][3] // 2)
        
        # 왼 눈, 오른 눈 순서 맞추기
        if x1 > x2:
            x1, y1, x2, y2 = x2, y2, x1, y1

        #cv2.circle(faceROI, (x1, y1), 5, (255, 0, 0), 2, cv2.LINE_AA)
        #cv2.circle(faceROI, (x2, y2), 5, (255, 0, 0), 2, cv2.LINE_AA)

        # 두 눈 사이의 거리를 이용하여 스케일링 팩터를 계산 (두 눈이 수평하다고 가정)
        # 실제 입력 영상에서 두 눈의 좌표 간격, 안경 영상에서 두 눈의 좌표 간격을 이용하여 resize factor 지정
        fx = (x2 - x1) / (ex2 - ex1)
        glasses2 = cv2.resize(glasses, (0, 0), fx=fx, fy=fx, interpolation=cv2.INTER_AREA)

        # 크기 조절된 안경 영상을 합성할 위치 계산 (좌상단 좌표)
        pos = (x1 - int(ex1 * fx), y1 - int(ey1 * fx))

        # 영상 합성
        overlay(frame, glasses2, pos)

    # 프레임 저장 및 화면 출력
    out.write(frame)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [44]:
frame.shape

(480, 640, 3)

In [49]:
frame[...,0]

array([[181, 182, 182, ..., 139, 139, 139],
       [180, 181, 182, ..., 138, 138, 138],
       [181, 182, 182, ..., 136, 136, 137],
       ...,
       [ 87,  87,  87, ..., 100, 100,  99],
       [ 87,  88,  88, ..., 100,  99,  99],
       [ 87,  88,  89, ..., 100,  99,  98]], dtype=uint8)

In [53]:
frame[...,0].shape

(480, 640)

In [50]:
frame[...,1]

array([[182, 183, 183, ..., 135, 135, 135],
       [182, 183, 184, ..., 136, 136, 136],
       [183, 184, 184, ..., 137, 137, 137],
       ...,
       [ 85,  85,  84, ...,  98,  98,  97],
       [ 85,  86,  85, ...,  98,  97,  97],
       [ 85,  86,  86, ...,  98,  97,  96]], dtype=uint8)

In [51]:
frame[...,2]

array([[173, 174, 174, ..., 140, 140, 140],
       [173, 174, 175, ..., 139, 139, 139],
       [174, 175, 175, ..., 138, 138, 140],
       ...,
       [ 75,  75,  76, ...,  82,  82,  81],
       [ 75,  76,  77, ...,  82,  81,  81],
       [ 75,  76,  78, ...,  82,  81,  80]], dtype=uint8)

In [52]:
frame[..., 3]

IndexError: index 3 is out of bounds for axis 2 with size 3